In [24]:
from openai import OpenAI
from vroom.NER import *
import os
from vroom.GPTTokenizer import GPTTokenizer
import json
from vroom.GraphManager import GraphManager
from vroom.baseline import get_cooccurences_with_aliases

unlabeled_chapter = os.path.join("../", "data", "test_set", "prelude_a_fondation", "chapter_1.unlabeled")

entities, chunks = get_entities_from_file(unlabeled_chapter)

all_entities_names = []
for chunk in entities: 
    all_entities_names += [
        entity["word"] for entity in chunk
    ]
entities = set(all_entities_names)

text_tagged = tag_text_with_entities(unlabeled_chapter, entities)

# Appel à l'API de GPT-3 pour générer les alias
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')
if not OPENAI_API_KEY:
    raise ValueError("La clé API OpenAI n'est pas définie dans les variables d'environnement.")

system_prompt = """
Tu es un expert dans l'étude des personnages dans les romans. J'ai un texte annoté par un modèle d'NER, et je n'ai gardé que les entités qu'il a estimé être des personnages. Elles sont situées entre les balises <PER></PER>. J'ai besoin qu'à partir de ce texte, tu me construises un JSON qui compile tous les personnages et leurs alias associé. Tu dois le faire comme dans cet exemple : 

Input : 
<start>
Mathématicien  <PER> CLÉON Ier </PER>  — ... dernier  <PER> Empereur </PER>  galactique de la dynastie Entun. Né en l'an 11988 de l'Ère Galactique, la même année que  <PER> Hari  Seldon  </PER> . (On pense que la date de naissance de  <PER> Seldon </PER> , que certains estiment douteuse, aurait pu être « ajustée » pour coïncider avec celle de  <PER> Cléon </PER>  que  <PER> Seldon </PER>
<end>

Output : 
<start>
{
   " CLÉON Ier ":{
      "aliases":[
         "Cléon Ier",
         "Cléon",
         "Empereur"
      ]
   },
   "Hari Seldon":{
      "aliases":[
         "Hari Seldon",
         "Seldon"
      ]
   }
}
<end>

Pour chaque entrée dans le JSON, tu devras extraire un seul personnage du texte, et y associer tout ses alias. Rajoute la clé de chaque personnage dans ses alias.

Voici l'input :
"""

client = OpenAI(api_key=OPENAI_API_KEY)

model = "gpt-3.5-turbo-1106" # gpt-4-1106-preview

len_chunk = 5000
tokenizer = GPTTokenizer(model)
len_prompt = len(tokenizer.tokenize(system_prompt))

chunks = chunk_text(text_tagged, len_chunk)

json_entities = []

for chunk in chunks:
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": chunk},
        ],
        seed=42,
        temperature=0,
        response_format={ "type": "json_object" },
        )

    json_entities.append(response.choices[0].message.content)
    print("Tokens sent : ", len(tokenizer.tokenize(chunk+system_prompt)))
    print("Tokens received : ", len(tokenizer.tokenize(response.choices[0].message.content)))
    print("Text computed : ", chunk)
    print("Text received : ", response.choices[0].message.content)



Tokens sent :  1946
Tokens received :  164
Text computed :  Mathématicien <PER> CLÉON Ier </PER> — ... dernier <PER> Empereur </PER> galactique de la dynastie Entun. Né en l’an 11988 de l’Ère Galactique, la même année que <PER> Hari Seldon </PER> . (On pense que la date de naissance de <PER> Seldon </PER> , que certains estiment douteuse, aurait pu être « ajustée » pour coïncider avec celle de <PER> Cléon </PER> que <PER> Seldon </PER> , peu après son arrivée sur <PER> Trantor </PER> , est censé avoir rencontré.) <PER> Cléon </PER> est monté sur le trône impérial en 12010, à l’âge de vingt-deux ans, et son règne représente un étrange intervalle de calme dans ces temps troublés. Cela est dû sans aucun doute aux talents de son chef d’état-major, <PER> Eto Demerzel </PER> , qui sut si bien se dissimuler à la curiosité médiatique que l’on a fort peu de renseignements à son sujet. <PER> Cléon </PER> , quant à lui... ENCYCLOPAEDIA GALACTICA2 Étouffant un léger bâillement, <PER> Cléon </PER> 

In [25]:
import json

# Dictionnaire global pour le résultat fusionné
merged_entities = {}

for entity_str in json_entities:
    # Convertir la chaîne JSON en dictionnaire
    entity_dict = json.loads(entity_str)

    for key, value in entity_dict.items():
        if key not in merged_entities:
            # Si la clé n'existe pas, l'ajouter au dictionnaire global
            merged_entities[key] = value
        else:
            # Si la clé existe déjà, fusionner ou remplacer selon vos besoins
            # Par exemple, fusionner les listes d'alias
            existing_aliases = set(merged_entities[key]['aliases'])
            new_aliases = set(value['aliases'])
            merged_aliases = list(existing_aliases.union(new_aliases))
            merged_entities[key]['aliases'] = merged_aliases

# Convertir le dictionnaire global en chaîne JSON
merged_json = json.dumps(merged_entities, indent=4, ensure_ascii=False)
print(merged_json)


{
    "CLÉON Ier": {
        "aliases": [
            "CLÉON Ier",
            "Cléon Ier",
            "Empereur",
            "Cléon",
            "Empereur galactique"
        ]
    },
    "Hari Seldon": {
        "aliases": [
            "Hari Seldon",
            "professeur",
            "Seldon"
        ]
    },
    "Eto Demerzel": {
        "aliases": [
            "Eto Demerzel",
            "Demerzel"
        ]
    },
    "Sire": {
        "aliases": [
            "Sire"
        ]
    },
    "Lieutenant Alban Wellis": {
        "aliases": [
            "Lieutenant Alban Wellis",
            "Wellis"
        ]
    },
    "Empereur": {
        "aliases": [
            "Cléon",
            "Empereur",
            "Cléon Ier",
            "Sire"
        ]
    },
    "Seldon": {
        "aliases": [
            "Hari Seldon",
            "Seldon"
        ]
    },
    "Cléon": {
        "aliases": [
            "Cléon",
            "Empereur",
            "Cléon Ier"
        ]
    

In [27]:

filtering_prompt = """
Tu es un expert dans l'étude des personnages dans les romans. J'ai un texte annoté par un modèle d'NER, et je n'ai gardé que les entités qu'il a estimé être des personnages.
Nous avons eu ce JSON comme résultat mais il se peut qu'il y ait des doublons. Tu dois donc les supprimer. Si tu vois qu'un personnage a un alias qui est aussi un personnage, privilégie l'alias au personnage. Tu as stricte interdiction d'inventer des alias ou des personnages.

Voici un exemple :
Voici l'input :
<start>
{
    " CLÉON Ier ":{
        "aliases":[
            "Cléon Ier",
            "Cléon",
            "Empereur",
            "CLÉON Ier"
        ]
    },
    "Hari Seldon":{
        "aliases":[
            "Hari Seldon",
            "Seldon"
        ]
    }
    "Empereur":{
        "aliases":[
            "Empereur"
        ]
    }
    "Seldon":{
        "aliases":[
            "Seldon"
        ]
    }
}
<end>

Output :
<start>
{
    " CLÉON Ier ":{
        "aliases":[
            "Cléon Ier",
            "Cléon",
            "Empereur"
        ]
    },
    "Hari Seldon":{
        "aliases":[
            "Hari Seldon",
            "Seldon"
        ]
    }
}
<end>
"""

response = client.chat.completions.create(
    model=model,
    messages=[
        {"role": "system", "content": filtering_prompt},
        {"role": "user", "content": merged_entities.__str__()},
    ],
    seed=42,
    temperature=0,
    response_format={ "type": "json_object" },
    )

print("Tokens sent : ", len(tokenizer.tokenize(system_prompt)))
print("Tokens received : ", len(tokenizer.tokenize(response.choices[0].message.content)))
print("Text received : ", response.choices[0].message.content)


Tokens sent :  378
Tokens received :  171
Text received :  {
    "CLÉON Ier": {
        "aliases": ["CLÉON Ier", "Cléon Ier", "Empereur", "Cléon"]
    },
    "Hari Seldon": {
        "aliases": ["Hari Seldon", "Seldon"]
    },
    "Eto Demerzel": {
        "aliases": ["Eto Demerzel", "Demerzel"]
    },
    "Sire": {
        "aliases": ["Sire", "Demerzel"]
    },
    "Lieutenant Alban Wellis": {
        "aliases": ["Lieutenant Alban Wellis", "Wellis"]
    },
    "Trantor": {
        "aliases": ["Trantor"]
    },
    "Hummin": {
        "aliases": ["Hummin"]
    }
}
{'entity_group': 'PER', 'word': 'CLÉON Ier', 'start': 0, 'end': 0}


TypeError: string indices must be integers, not 'str'

In [38]:

# Ecrire le json dans un fichier
with open("../data/test_set/prelude_a_fondation/chapter_1.gpt.json", "w") as f:
    json_entities = json.loads(response.choices[0].message.content)
    json.dump(json_entities, f, indent=4, ensure_ascii=False)

print(json_entities)


{'CLÉON Ier': {'aliases': ['CLÉON Ier', 'Cléon Ier', 'Empereur', 'Cléon']}, 'Hari Seldon': {'aliases': ['Hari Seldon', 'Seldon']}, 'Eto Demerzel': {'aliases': ['Eto Demerzel', 'Demerzel']}, 'Sire': {'aliases': ['Sire', 'Demerzel']}, 'Lieutenant Alban Wellis': {'aliases': ['Lieutenant Alban Wellis', 'Wellis']}, 'Trantor': {'aliases': ['Trantor']}, 'Hummin': {'aliases': ['Hummin']}}


In [66]:
import nltk

def cooccurences_from_gpt_json(text, entities):
    # Tokeniser le texte
    tokens = nltk.word_tokenize(text)
    # Créer un dictionnaire pour stocker les positions de chaque alias
    alias_positions = {}

    # Parcourir les entités pour trouver les positions de leurs alias
    for _, info in json_entities.items():
        aliases = info['aliases']
        for alias in aliases:
            alias_words = nltk.word_tokenize(alias)
            alias_length = len(alias_words)

            # Parcourir les tokens pour trouver les occurrences de l'alias
            for i in range(len(tokens) - alias_length + 1):
                if tokens[i:i + alias_length] == alias_words:
                    # Ajouter la position de début de l'alias dans le texte
                    alias_positions.setdefault(alias, []).append(i)

    cooccurrences = []
    for entity1, aliases1 in json_entities.items():
        for entity2, aliases2 in json_entities.items():
            if entity1 != entity2:
                # Vérifiez chaque combinaison d'alias pour la cooccurrence
                for alias1 in aliases1['aliases'] + [entity1]:
                    for alias2 in aliases2['aliases'] + [entity2]:
                        if alias1 in alias_positions and alias2 in alias_positions:
                            if any(abs(pos1 - pos2) <= 25 for pos1 in alias_positions[alias1] for pos2 in alias_positions[alias2]):
                                # Ajouter la paire de listes d'alias à la liste de cooccurrences
                                cooccurrences.append((aliases1['aliases'], aliases2['aliases']))
                                print("Cooccurrence trouvée : ", (aliases1['aliases'], aliases2['aliases'], "A l'indice : ", tokens.index(alias1)))
                                break  # Arrêtez de chercher d'autres occurrences pour cette paire
    return cooccurrences

In [71]:
import html
graph_manager = GraphManager()

# Récupérer le texte original
with open(unlabeled_chapter, "r") as f:
    text = f.read()

cooccurences = cooccurences_from_gpt_json(text, json_entities)

print(cooccurences)

graph_manager.add_cooccurrences(cooccurences)

graph = graph_manager.generate_graph()
print("".join(html.unescape(s) if isinstance(s, str) else s for s in graph))
graph_manager.save_graph_to_graphml("chapter_1.graphml")

Cooccurrence trouvée :  (['CLÉON Ier', 'Cléon Ier', 'Empereur', 'Cléon'], ['Hari Seldon', 'Seldon'])
Cooccurrence trouvée :  (['CLÉON Ier', 'Cléon Ier', 'Empereur', 'Cléon'], ['Hari Seldon', 'Seldon'])
Cooccurrence trouvée :  (['CLÉON Ier', 'Cléon Ier', 'Empereur', 'Cléon'], ['Eto Demerzel', 'Demerzel'])
Cooccurrence trouvée :  (['CLÉON Ier', 'Cléon Ier', 'Empereur', 'Cléon'], ['Eto Demerzel', 'Demerzel'])
Cooccurrence trouvée :  (['CLÉON Ier', 'Cléon Ier', 'Empereur', 'Cléon'], ['Sire', 'Demerzel'])
Cooccurrence trouvée :  (['CLÉON Ier', 'Cléon Ier', 'Empereur', 'Cléon'], ['Sire', 'Demerzel'])
Cooccurrence trouvée :  (['CLÉON Ier', 'Cléon Ier', 'Empereur', 'Cléon'], ['Trantor'])
Cooccurrence trouvée :  (['CLÉON Ier', 'Cléon Ier', 'Empereur', 'Cléon'], ['Trantor'])
Cooccurrence trouvée :  (['Hari Seldon', 'Seldon'], ['CLÉON Ier', 'Cléon Ier', 'Empereur', 'Cléon'])
Cooccurrence trouvée :  (['Hari Seldon', 'Seldon'], ['CLÉON Ier', 'Cléon Ier', 'Empereur', 'Cléon'])
Cooccurrence trouvée :